# Assembling reads into contigs

This notebook will go through the workflow for using the metaspades and megahit assembly tools. In this section we are going to assemble our reads into contigs. Contigs are fragments of DNA that represent parts of a genome. If you are lucky, you might even be able to assemble an entire genome in a single contig! But, most of the time, contigs are just part of a genome with missing fragments in between contigs that prevent you from assembling the entire genome.

1. An introduction to [Metaspades](https://cab.spbu.ru/files/release3.12.0/manual.html)


## Getting Started

You will need to rerun this section each time you come back to this notebook to reset all directories and variables.

In [1]:
# set the variables for your netid and xfile
netid = "kolodisner"
xfile = "xac"

In [2]:
# Go into the working directory
work_dir = "/xdisk/bhurwitz/virus_hunting/" + netid + "/04_assembly"
%cd $work_dir

/xdisk/bhurwitz/virus_hunting/kolodisner/04_assembly


## Creating a config file
The scripts below executes code that requires certain variables to be set. So we don't need to edit the code in the script, we are going to use a config file that defines all of these variables for us. Then when we want to use these variables in the script, we will "source" the config file to set the variables.

### Data Management

We'll be creating two assemblies based on the trimmed/human removed reads. Let's setup the output directories ahead of time.

In [4]:
!mkdir $work_dir/out_spades

In [5]:
# create a config file with all of the variables you need
# notice that we will assemble the reads that are both trimmed and have human removed.
!echo "export NETID=$netid" > config.sh
!echo "export XFILE=$xfile" >> config.sh
!echo "export XFILE_DIR=/xdisk/bhurwitz/virus_hunting/$netid" >> config.sh
!echo "export FASTQ_DIR=/xdisk/bhurwitz/virus_hunting/$netid/03_contam_removal" >> config.sh
!echo "export OUT_SPADES=/xdisk/bhurwitz/virus_hunting/$netid/04_assembly/out_spades" >> config.sh

In [6]:
!cat config.sh

export NETID=kolodisner
export XFILE=xac
export XFILE_DIR=/xdisk/bhurwitz/virus_hunting/kolodisner
export FASTQ_DIR=/xdisk/bhurwitz/virus_hunting/kolodisner/03_contam_removal
export OUT_SPADES=/xdisk/bhurwitz/virus_hunting/kolodisner/04_assembly/out_spades


## Step 1: Running Metaspades to create contigs

You will be assembling your reads using a program called spades, that has the metaspades.py program within it for assembling metagenomes comprised of multiple organisms.

It's important to note that this assembler is memory intensive, and for large files it takes a lot of resource and time. A common error for large files is running out of memory to complete the job in the HPC. If needed, we can modify our script if it requires more memory. 

Puma can have 94 CPUs @ 5gb/CPU <br>
Ocelote can have 28 CPUs @ 6gb/CPU

This [HPC documentation](https://public.confluence.arizona.edu/display/UAHPC/Running+Jobs+with+SLURM) is handy to have as you edit your scripts and use different HPCs within UA.

In [7]:
# Create a script to run metaspades
# A few important points:
# 1. We are using the variables from the config file via the `source ./config.sh` command in the script.
# 2. metaspades runs on each of the fastq files in the trimmed $FASTQ_DIR
# 3. The results will be written into our $OUT_SPADES directory
# 4. Notice that we are asking for alot more resource (28 cores and 5G of memory per core), we are also asking for more time (24 hours)
my_code = '''#!/bin/bash
#SBATCH --output=Job-spades-%a.out
#SBATCH --account=bhurwitz
#SBATCH --partition=standard
#SBATCH --nodes=1
#SBATCH --ntasks=1
#SBATCH --time=48:00:00
#SBATCH --cpus-per-task=28
#SBATCH --mem-per-cpu=5gb
#SBATCH --array=0-2

pwd; hostname; date

source ./config.sh
names=($(cat $XFILE_DIR/$XFILE))

SAMPLE_ID=${names[${SLURM_ARRAY_TASK_ID}]}

PAIR1=${FASTQ_DIR}/${SAMPLE_ID}_1.fastq*
PAIR2=${FASTQ_DIR}/${SAMPLE_ID}_2.fastq*

#add threads flag & exposition on adding threads or it runs inefficient
apptainer run /contrib/singularity/shared/bhurwitz/spades:3.15.5--h95f258a_1.sif metaspades.py \
   -o ${OUT_SPADES}/${names[${SLURM_ARRAY_TASK_ID}]} \
   --pe1-1 ${PAIR1} \
   --pe1-2 ${PAIR2}
'''

with open('run_metaspades.sh', mode='w') as file:
    file.write(my_code)

In [8]:
# you should be in your working directory when you run this script
# do you see your config.sh file, and the rrun_metaspades.sh script?
!pwd
!ls

/xdisk/bhurwitz/virus_hunting/kolodisner/04_assembly
config.sh  out_spades  run_metaspades.sh


In [9]:
# Let's run sbatch to run metaspades on each of the FASTQ files
# Remember that this may take a while to run, so take a break, 
# and get a coffee.
!sbatch ./run_metaspades.sh

Submitted batch job 3036763


In [6]:
# You can check if it is running using the squeue command
# Check for all jobs under your netid
!squeue --user=$netid

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
         3036763_0  standard run_meta kolodisn  R    9:11:03      1 i1n14
         3036763_1  standard run_meta kolodisn  R    9:11:03      1 i4n15
         3036763_2  standard run_meta kolodisn  R    9:11:03      1 i6n17
           3040126  standard  jupyter kolodisn  R       3:53      1 i16n3


In [4]:
# Once your jobs have run (or are running) you can check the progress
# and also look for errors in the *out files
# For example, you can look at Job-spades-0.out
!ls
!cat Job-spades-0.out

Job-spades-0.out  Job-spades-2.out  out_spades
Job-spades-1.out  config.sh	    run_metaspades.sh
/xdisk/bhurwitz/virus_hunting/kolodisner/04_assembly
i1n14.ocelote.hpc.arizona.edu
Sat Mar 30 12:58:20 MST 2024
Command line: /usr/local/bin/metaspades.py	-o	/xdisk/bhurwitz/virus_hunting/kolodisner/04_assembly/out_spades/ERR2198703	--pe1-1	/xdisk/bhurwitz/virus_hunting/kolodisner/03_contam_removal/ERR2198703_1.fastq.gz	--pe1-2	/xdisk/bhurwitz/virus_hunting/kolodisner/03_contam_removal/ERR2198703_2.fastq.gz	

System information:
  SPAdes version: 3.15.5
  Python version: 3.10.6
  OS: Linux-3.10.0-1160.108.1.el7.x86_64-x86_64-with-glibc2.28

Output dir: /xdisk/bhurwitz/virus_hunting/kolodisner/04_assembly/out_spades/ERR2198703
Mode: read error correction and assembling
Debug mode is turned OFF

Dataset parameters:
  Metagenomic mode
  Reads:
    Library number: 1, library type: paired-end
      orientation: fr
      left reads: ['/xdisk/bhurwitz/virus_hunting/kolodisner/03_contam_removal/ERR

In [5]:
# Do you see a contigs file?
!ls $work_dir/out_spades/*

/xdisk/bhurwitz/virus_hunting/kolodisner/04_assembly/out_spades/ERR2198703:
K21  configs	   input_dataset.yaml  pipeline_state	spades.log
K33  corrected	   misc		       run_spades.sh	tmp
K55  dataset.info  params.txt	       run_spades.yaml

/xdisk/bhurwitz/virus_hunting/kolodisner/04_assembly/out_spades/ERR2198704:
K21  configs	   input_dataset.yaml  pipeline_state	spades.log
K33  corrected	   misc		       run_spades.sh	tmp
K55  dataset.info  params.txt	       run_spades.yaml

/xdisk/bhurwitz/virus_hunting/kolodisner/04_assembly/out_spades/ERR2198705:
K21  configs	   input_dataset.yaml  pipeline_state	spades.log
K33  corrected	   misc		       run_spades.sh	tmp
K55  dataset.info  params.txt	       run_spades.yaml


Great job! You should now have assemblies from metaspades. You can kick off the assembly below with megahit without disrupting your work from above. Go for it!

## Final Step
Copy your notebook to the current working directory

In [ ]:
cp ~/04_assembly.ipynb $work_dir